# Tarea 5

In [1]:
# librerias
import os, mlflow
from dotenv import load_dotenv
import pickle
import pandas as pd
from sklearn.metrics import  root_mean_squared_error
from sklearn.feature_extraction import  DictVectorizer

In [2]:
load_dotenv(override=True)
EXPERIMENT_NAME = "/Users/sarahbeltrang@gmail.com/nyc-taxi-experiments"
mlflow.set_tracking_uri("databricks")
mlflow.set_experiment(experiment_name=EXPERIMENT_NAME)

<Experiment: artifact_location='dbfs:/databricks/mlflow-tracking/86265904456551', creation_time=1761101109923, experiment_id='86265904456551', last_update_time=1761103332360, lifecycle_stage='active', name='/Users/sarahbeltrang@gmail.com/nyc-taxi-experiments', tags={'mlflow.experiment.sourceName': '/Users/sarahbeltrang@gmail.com/nyc-taxi-experiments',
 'mlflow.experimentType': 'MLFLOW_EXPERIMENT',
 'mlflow.ownerEmail': 'sarahbeltrang@gmail.com',
 'mlflow.ownerId': '76150295292949'}>

In [3]:
def read_dataframe(filename):

    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [4]:
df_train = read_dataframe("C:/Users/sarah/apps/sem_5/pcd/nyc-taxi-predictions-2025/data/green_tripdata_2025-01.parquet")
df_val = read_dataframe("C:/Users/sarah/apps/sem_5/pcd/nyc-taxi-predictions-2025/data/green_tripdata_2025-02.parquet")

In [5]:
def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)